<a href="https://colab.research.google.com/github/Kunj1/Digit-classifier/blob/main/Copy_of_digit_recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.7/286.7 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
import gradio as gr

In [ ]:
(X_train,y_train),(X_test,y_test)=mnist.load_data()

In [ ]:
DigitClassifier=Sequential()
DigitClassifier.add(Conv2D(32,(5,5),input_shape=(28,28,1),activation='relu'))
DigitClassifier.add(MaxPooling2D(pool_size=(2,2)))
DigitClassifier.add(Flatten())
DigitClassifier.add(Dense(128,activation='relu'))
DigitClassifier.add(Dense(10,activation='softmax'))

DigitClassifier.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
Xm_train=X_train.reshape((X_train.shape[0],28,28,1)).astype('float32')
Xm_test=X_test.reshape((X_test.shape[0],28,28,1)).astype('float32')
Xm_train=Xm_train/255
Xm_test=Xm_test/255

In [ ]:
ym_train=np_utils.to_categorical(y_train)
ym_test=np_utils.to_categorical(y_test)

In [ ]:
DigitClassifier.fit(Xm_train,ym_train,validation_data=(Xm_test,ym_test),epochs=5,batch_size=100)

In [ ]:
scores=DigitClassifier.evaluate(Xm_test,ym_test)

In [ ]:
def output_class(image):
  image = np.expand_dims(image,axis=0)
  R1 = DigitClassifier.predict(image)
  R3 = np.argmax(R1,axis=1)
  return str(R3[0])

In [ ]:
def recognize_digit(image):
    image = image.reshape(1, -1)  # add a batch dimension
    prediction = DigitClassifier.predict(image).tolist()[0]
    return {str(i): prediction[i] for i in range(10)}

output_component = gr.outputs.Label()

gr.Interface(fn=output_class,
             inputs="sketchpad",
             outputs=output_component,
             title="MNIST Sketchpad",
             description="Draw a number 0 through 9 on the sketchpad, and click submit to see the model's predictions. Model trained on the MNIST",debug = True).launch()